In [45]:
import numpy as np
import torch 
from tqdm import tqdm,trange    
from torch import Tensor
from torch.utils.data import Dataset, DataLoader, random_split

import matplotlib.pyplot as plt
# from torch.utils.tensorboard import SummaryWriter

import os

print(os.getcwd())  



/home/alinjar/manipulator/manipulator_mujoco/flow_matching


In [46]:
from flow_model_manipulator import Flow
import open3d as o3d



In [47]:


torch.set_float32_matmul_precision('high')

# writer = SummaryWriter("./logs")
test = 'manipulator'

In [48]:
BATCH_SIZE      = 500
LEARNING_RATE   = 1e-3
SEED            = 0
DEVICE          = 'cuda'      
NUM_EPOCH       = 2000

In [49]:
torch.manual_seed(seed=SEED)
torch.cuda.manual_seed(seed=SEED)

In [ ]:
class TrajDataset(Dataset):
    def __init__(self, theta_init_data, thetadot_init_data, target_pos_data, target_quat_data, index_data):
        
        # goal
        self.theta_init = theta_init_data
        self.thetadot_init = thetadot_init_data
        self.target_pos = target_pos_data
        self.target_quat = target_quat_data


        self.index = index_data
        
    
    def __len__(self):
        return len(self.theta_init)    
            
    def __getitem__(self, idx):

        theta_init = self.theta_init[idx]
        thetadot_init = self.thetadot_init[idx]
        target_pos = self.target_pos[idx]
        target_quat = self.target_quat[idx]
        index = self.index[idx]
                 
        return torch.tensor(theta_init).float(), torch.tensor(thetadot_init).float(), torch.tensor(target_pos).float(), \
            torch.tensor(target_quat).float(), index

In [51]:
import os
import numpy as np

# One step behind the current working directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

print(parent_dir)


/home/alinjar/manipulator/manipulator_mujoco


In [52]:


# Build relative paths from parent directory
data_dir = os.path.join(parent_dir, "custom_data_target_0_inference_False")
pcd_dir = os.path.join(parent_dir, "pcd_data")

print(data_dir)


/home/alinjar/manipulator/manipulator_mujoco/custom_data_target_0_inference_False


In [53]:
# Load the data

theta_init_data = np.loadtxt(os.path.join(data_dir, "theta.csv"), delimiter=",")
thetadot_init_data = np.loadtxt(os.path.join(data_dir, "thetadot.csv"), delimiter=",")
target_pos_data = np.loadtxt(os.path.join(data_dir, "target_positions.csv"), delimiter=",")
target_quat_data = np.loadtxt(os.path.join(data_dir, "target_quaternions.csv"), delimiter=",")

index_data = np.arange(1, len(theta_init_data) + 1)


pcd_path = os.path.join(pcd_dir, "output_scene_without_robot_unpacked_rgb.pcd")
pcd = o3d.io.read_point_cloud(pcd_path)

points = np.asarray(pcd.points)       # XYZ points
colors = np.asarray(pcd.colors)       # RGB (if present)

# print(points.shape)
# print(colors.shape)

print(theta_init_data.shape)
print(thetadot_init_data.shape)
print(target_pos_data.shape)
print(target_quat_data.shape)


(2638, 6)
(2638, 6)
(2, 3)
(2, 4)


In [54]:
theta_init_mean, theta_init_std = torch.tensor(theta_init_data.mean()).to(DEVICE), torch.tensor(theta_init_data.std()).to(DEVICE)
thetadot_init_mean, thetadot_init_std = torch.tensor(thetadot_init_data.mean()).to(DEVICE), torch.tensor(thetadot_init_data.std()).to(DEVICE)
target_pos_mean, target_pos_std = torch.tensor(target_pos_data.mean()).to(DEVICE), torch.tensor(target_pos_data.std()).to(DEVICE)
target_quat_mean, target_quat_std = torch.tensor(target_quat_data.mean()).to(DEVICE), torch.tensor(target_quat_data.std()).to(DEVICE)


In [55]:
dataset = TrajDataset(theta_init_data, thetadot_init_data, target_pos_data, target_quat_data, index_data)
print(len(dataset))

train_size = int(0.9 * len(dataset))  
test_size = len(dataset) - train_size  

# Create a generator with a fixed seed
generator = torch.Generator().manual_seed(0)

train_dataset, test_dataset = random_split(dataset, [train_size, test_size], generator=generator)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=False)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=False)

out_chan = 512

flow = Flow(out_chan,theta_init_mean,theta_init_std, thetadot_init_mean, theta_init_std, target_pos_mean, target_pos_std,
            target_quat_mean, target_quat_std).cuda()

loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.AdamW(flow.parameters(), lr=LEARNING_RATE)

2638


In [56]:
flow.train()
c_flow = torch.compile(flow)

avg_losses = []
last_loss = torch.inf


for epoch in trange(NUM_EPOCH):
	losses = []
	
	for (theta_init,thetadot_init, target_pos, target_quat, index) in train_loader:


		theta_init = theta_init.to(DEVICE)
		thetadot_init = thetadot_init.to(DEVICE)
		target_pos = target_pos.to(DEVICE)
		target_quat = target_quat.to(DEVICE)

		motion_data = [theta_init, thetadot_init, target_pos, target_quat]
		x_1 = torch.stack(motion_data)
		x_0 = torch.randn_like(x_1)
		t = torch.rand(len(x_1), 1, 1).to(device=DEVICE)
		x_t = (1 - t) * x_0 + t * x_1
		dx_t = x_1 - x_0
	
		loss = loss_fn(c_flow(x_t, motion_data, t, pcd), dx_t)
		losses.append(loss.item())

		optimizer.zero_grad()
		loss.backward()
		optimizer.step()


	mean_loss = np.mean(losses)
	avg_losses.append(mean_loss)
	# scheduler.step()

	if epoch % 50 == 0:
		print(f"Epoch: {epoch + 1}, Train Loss: {mean_loss:.3f}")
	#writer.add_scalar('test_{}'.format(test), loss, epoch)

	if loss <= last_loss:
		torch.save(flow.state_dict(), f"./weights/test_{test}_lowest.pt")
		torch.save(optimizer.state_dict(), f"./opts/test_{test}_lowest.pt")
		last_loss = loss

  0%|          | 0/2000 [00:00<?, ?it/s]


IndexError: index 1769 is out of bounds for axis 0 with size 2